Привет, меня зовут Артем Хуршудов. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ...
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

### <font color='orange'>Общее впечатление</font>
* Большое спасибо за проделанную работу. Видно, что приложено много усилий.
* Выводы и рассуждения получились содержательными, их было интересно читать.
* Радует, что тебе удалось разобраться с техникой бутстрапа.
* Отлично, что доверительный интервал и риск убытков посчитаны верно.
* Редко, кто сдает эту работу с первого раза, молодец!
* Проект может быть зачтен, но я его отправлю назад, чтобы у тебя была возможность задать вопросы и внести правки, при желании. Однако, ты можешь просто вернуть проект в таком же виде и я его зачту.

### <font color='orange'>Общее впечатление (ревью 2)</font>
* Проект зачтен!
* Удачи в дальнейшем обучении и следующих работах!

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Условия задачи:  
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Импортируем все необходимые библиотеки для решения данной задачи

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')

In [3]:
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
data_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [6]:
data_1 = pd.read_csv('/datasets/geo_data_1.csv')

In [7]:
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [8]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [10]:
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [11]:
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [12]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [13]:
data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


В трех представленных датасетах мы имеим следующие данные:
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

Пропущенные значения во всех трех таблицах отсутсвуют, но необходимо проверить данные на явные дубликаты.

In [14]:
print(data_0.duplicated().sum())
print(data_1.duplicated().sum())
print(data_2.duplicated().sum())

0
0
0


Повторяющиеся строки не обнаружены во всех трех таблицах.

<div class="alert alert-block alert-success">
<b>Успех:</b> Данные загружены корректно, первичный осмотр проведен.
</div>

Т.к. по условию задачи все три показателя важны, то нет смысла проверять их на корреляцию - выбрасывать данные из таблиц мы не будем. А так же, по скольку невозможно оценить что является вбросом, а что нет (мы не знаем что это за признаки и что для них считается нормальным значением), просто отмасштабируем необходимые для расчетов столбцы: f0, f1, f2.

Определим целевой признак и признаки.

In [15]:
def target_features(data):
    target = data['product']
    features= data.drop(['product', 'id'], axis=1)
    return target, features

In [16]:
target_0, features_0 = target_features(data_0)

In [17]:
target_1, features_1 = target_features(data_1)

In [18]:
target_2, features_2 = target_features(data_2)

In [19]:
def train_valid(features, target):
    features_train, features_valid, target_train, target_valid = train_test_split(
       features, target, test_size=0.25, random_state=12345)
    return features_train, features_valid, target_train, target_valid

In [20]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_valid(features_0, target_0)

In [21]:
print('Размер обучающей выборки:', features_train_0.shape, target_train_0.shape)
print('Размер валидационной выборки:', features_valid_0.shape, target_valid_0.shape)

Размер обучающей выборки: (75000, 3) (75000,)
Размер валидационной выборки: (25000, 3) (25000,)


In [22]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_valid(features_1, target_1)

In [23]:
print('Размер обучающей выборки:', features_train_1.shape, target_train_1.shape)
print('Размер валидационной выборки:', features_valid_1.shape, target_valid_1.shape)

Размер обучающей выборки: (75000, 3) (75000,)
Размер валидационной выборки: (25000, 3) (25000,)


In [24]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_valid(features_2, target_2)

In [25]:
print('Размер обучающей выборки:', features_train_2.shape, target_train_2.shape)
print('Размер валидационной выборки:', features_valid_2.shape, target_valid_2.shape)

Размер обучающей выборки: (75000, 3) (75000,)
Размер валидационной выборки: (25000, 3) (25000,)


Отмасштабируем признаки на обучающих и валидационных выборках.

In [26]:
#Пыталась записать функцию для масштабирования, но она не работает. В чем ошибка? 
def scaler_transforn(features_train, features_valid):
    numeric = ['f0', 'f1', 'f2']
    scaler = StandardScaler()
    scaler.fit(features_train[numeric])
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    
    return features_train, features_valid

In [29]:
features_train_0, features_valid_0 = scaler_transforn(features_train_0, features_valid_0)
pd.options.mode.chained_assignment = None

<div class="alert alert-block alert-info">
<b>Совет: </b> А можешь показать ошибку, что значит не работает? Может быть это из-за опечатки в названии?
</div>

**Функция работает, какая-то магия)

In [28]:
#numeric = ['f0', 'f1', 'f2']

In [31]:
#scaler = StandardScaler()
#scaler.fit(features_train_0[numeric])
#features_train_0[numeric] = scaler.transform(features_train_0[numeric])
#features_valid_0[numeric] = scaler.transform(features_valid_0[numeric])

#pd.options.mode.chained_assignment = None

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что скалер был обучен только на тренировочной части данных!
</div>

In [30]:
features_train_0.head()

,f0,f1,f2
27212,-0.544828,1.390264,-0.094959
7866,1.455912,-0.480422,1.209567
62041,0.260460,0.825069,-0.204865
70185,-1.837105,0.010321,-0.147634
82230,-1.299243,0.987558,1.273181


In [33]:
#scaler = StandardScaler()
#scaler.fit(features_train_1[numeric])
#features_train_1[numeric] = scaler.transform(features_train_1[numeric])
#features_valid_1[numeric] = scaler.transform(features_valid_1[numeric])

#pd.options.mode.chained_assignment = None

In [31]:
features_train_1, features_valid_1 = scaler_transforn(features_train_1, features_valid_1)
pd.options.mode.chained_assignment = None

In [32]:
features_train_1.head()

,f0,f1,f2
27212,-0.850855,0.624428,0.296943
7866,1.971935,1.832275,0.294333
62041,1.079305,0.170127,-0.296418
70185,-1.512028,-0.887837,-0.880471
82230,-1.804775,-0.718311,-0.293255


In [35]:
#scaler = StandardScaler()
#scaler.fit(features_train_2[numeric])
#features_train_2[numeric] = scaler.transform(features_train_2[numeric])
#features_valid_2[numeric] = scaler.transform(features_valid_2[numeric])

#pd.options.mode.chained_assignment = None

In [33]:
features_train_2, features_valid_2 = scaler_transforn(features_train_2, features_valid_2)
pd.options.mode.chained_assignment = None

In [34]:
features_train_2.head()

,f0,f1,f2
27212,-0.526160,0.776329,-0.400793
7866,-0.889625,-0.404070,-1.222936
62041,-1.133984,0.208576,0.296765
70185,1.227045,1.570166,-0.764556
82230,-0.194289,0.878312,0.840821


В ходе работы над данным пунктом данные были отмасштабированы и подготовлены для дальнейшего обучения модели.

## Обучение и проверка модели

Для прогнозирования и расчетов будем использовать модель линейной регрессии

In [37]:
model = LinearRegression()

In [59]:
def RSME_data(features_train, target_train, features_valid, target_valid):
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = mean_squared_error(target_valid, predictions_valid)**0.5
    mean_valid = predictions_valid.mean()
    predictions = predictions_valid
    return print('RSME модели равно:', round(result, 2)), print('Средний запас предсказанного сырья равен:', round(mean_valid, 2)), predictions 

Применим функцию ко всем регионам

In [60]:
result_0, mean_valid_0, predictions_0 = RSME_data(features_train_0, target_train_0, features_valid_0, target_valid_0)

RSME модели равно: 37.58
Средний запас предсказанного сырья равен: 92.59


In [61]:
result_1, mean_valid_1, predictions_1 = RSME_data(features_train_1, target_train_1, features_valid_1, target_valid_1)

RSME модели равно: 0.89
Средний запас предсказанного сырья равен: 68.73


In [62]:
result_2, mean_valid_2, predictions_2 = RSME_data(features_train_2, target_train_2, features_valid_2, target_valid_2)

RSME модели равно: 40.03
Средний запас предсказанного сырья равен: 94.97


Для каждго региона был предсказан средний объем сырья:  
- 92.59;
- 68.73;
- 94.97.  

Наивысшее качество предсказания у модели линейной регресии во втором регионе (RSME = 0.89), однако и запас предсказанного сырья во вотором регине самый низкий, поэтому однозначно выбрать какой-либо регион для разработки на этом этапе не представляется возможным 

<div class="alert alert-block alert-success">
<b>Успех:</b> Модели обучены корректно. Радует, что ты активно используешь функции. Это помогает избежать дублирования кода, а значит уменьшает вероятность совершить ошибку/опечатку!
</div>

## Подготовка к расчёту прибыли

Бюджет на разработку скважин в регионе — 10 млрд рублей.

In [47]:
BUGET = 10000000000

При нынешних ценах один баррель сырья приносит 450 рублей дохода.  
Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

In [48]:
PROFIT_ONE_BARR = 450000

<div class="alert alert-block alert-info">

<b>Совет: </b> В названиях константных переменных лучше использовать только БОЛЬШИЕ буквы. Это соглашение между программистами: https://pythonworld.ru/osnovy/pep-8-rukovodstvo-po-napisaniyu-koda-na-python.html#id29 .
</div>

In [50]:
print('Необходимый общий объем добычи в регионе для безубыточности:', round(BUGET/PROFIT_ONE_BARR) )

Необходимый общий объем добычи в регионе для безубыточности: 22222


В регионе будут разрабатывать 200 точек, следовательно, каждая точка должна приносить:

In [76]:
POINTS = 200

In [77]:
round((BUGET/PROFIT_ONE_BARR)/points)

111

Ни один из регионов не обладает необходимым предсказанным запасом сырья в 111 баррелей нефти с 1 точки. Это говорит о том, что мы не можем наугад выбрать любую точку из всех трех таблиц и начать их бурить, т.к. высока вероятность того, что это будет убыточно.  
Следовательно, необходимо из всей массы точек выбрать лучшие и спрогнозировать средний объем добычи для них.

<div class="alert alert-block alert-success">
<b>Успех:</b> Точка безубыточности найдена верно, сравнение проведено!
</div>

## Расчёт прибыли и рисков 

Напишем функцию для расчёта прибыли по выбранным 200 скважинам и предсказаниям модели.

In [78]:
def profit(predictions, target, best_points):
    predictions = pd.Series(predictions, index=target.index)
    predictions_points = predictions.sort_values(ascending=False)
    target_points = target[predictions_points.index][:best_points]
    return sum(target_points)*PROFIT_ONE_BARR - BUGET

<div class="alert alert-block alert-success">
<b>Успех:</b> Функция для подсчета прибыли определена верно.
</div>

Напишем функцию для бустрап

Доверительный интервал равен 95%

In [82]:
def bootstrap(predictions, target, best_points):
    state = np.random.RandomState(0)
    values = []
    
    for i in range(1000):
        target_subsamples = target.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
        predictions_subsamples = predictions[target_subsamples.index]
        values.append(profit(predictions_subsamples, target_subsamples, best_points))

    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean_profit = values.mean()
    loss = np.mean(values < 0)

    return print('Средняя прибыль равна:', round(mean_profit, 2)), print('2.5%-квантиль:', lower), print('97.5%-квантиль:', upper), print('Риск убытков, %:', loss*100) 

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что для подсчета доверительного интервала использовался именно метод квантилей.
</div>

<div class="alert alert-block alert-success">
<b>Успех:</b> Риск найден очень элегантно.
</div>

Расчет для первого региона:

In [83]:
bootstrap(predictions_0, target_valid_0, POINTS)

Средняя прибыль равна: 410303327.13
2.5%-квантиль: -163033337.41302997
97.5%-квантиль: 972213446.6647601
Риск убытков, %: 7.7


(None, None, None, None)

Расчет для второго региона:

In [84]:
bootstrap(predictions_1, target_valid_1, POINTS)

Средняя прибыль равна: 507552476.62
2.5%-квантиль: 87320140.88688523
97.5%-квантиль: 915139275.2721876
Риск убытков, %: 0.6


(None, None, None, None)

Расчет для третьего региона:

In [85]:
bootstrap(predictions_2, target_valid_2, POINTS)

Средняя прибыль равна: 412263351.93
2.5%-квантиль: -131311003.10993758
97.5%-квантиль: 947922004.8609216
Риск убытков, %: 7.5


(None, None, None, None)

Самый минимальный риск получения убытка от разработки новых месторождений показал второй регион: риск равен 0.6%. Прогнозируемая прибыль от разработки 200 скважен во втором регионе составила 507552476.62 тыс.рублей.   
Средняя прибыль от разработки в этом регионе с 95% вероятностью будет находиться в диапазоне от 87320140.88688523 до 915139275.2721876 тыс.рублей.

<div class="alert alert-block alert-success">
<b>Успех:</b> Все статистики посчитаны правильно. С выбором региона согласен.
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован

В ходе проведения эксперимента мы сделали следующие выводы:
- данные были отмасштабированы для дальнейших расчетов;
- невозможно взять один из трех предложенных регионов и начать наугад разрабатывать 200 скважин, поэтому нам необходим ML для выбора оптимального региона и лучших точек для разработки;
- с помощью бустрепа нам удалось выделить регион с наименьшими рисками для разработки. Это регион под номер два (data_1). Его и предлагаю взять для дальнейшей разработки новых месторождений. Выручка от разработки лучших 200 точек будет лежать в диапазоне [87320141 : 915139275] тысяч рублей. 